In [31]:
import pandas as pd       
import os 
import math 
import numpy as np
import matplotlib.pyplot as plt
import librosa
import torch
from torch.utils.data import Dataset, DataLoader
from pathlib import Path

In [48]:
class MFCCDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir

    def __len__(self):
        return len(os.listdir(root_dir)) * 2

    def __getitem__(self, idx):
        
        noise = False
        num_files = len(os.listdir(self.root_dir))
        
        data_id = idx
        
        if idx >= num_files:
            data_id = idx - num_files
            noise = True
            
        
        f = os.listdir(self.root_dir)[data_id]
        y, sr = librosa.load(self.root_dir / f)
        m = librosa.feature.mfcc(y=y, sr=sr)
        
        
        if noise:
            n = np.random.normal(0,1, m.shape)
            m += n
        
        m_standardized = np.zeros(m.shape)
        for b in range(m.shape[0]):
            m_slice = m[b,:]
            centered = m_slice - np.mean(m_slice)
            if np.std(centered) != 0:
                centered_scaled = centered / np.std(centered)
                
            m_standardized[b,:] = centered_scaled
            
        delta1 = librosa.feature.delta(m_standardized, order=1)
        delta2 = librosa.feature.delta(m_standardized, order=2)
        mfcc_data = np.stack((m_standardized,delta1,delta2))
        
        return torch.tensor(mfcc_data)

In [49]:
train_data_dir = Path('C:/Users/omar_/Documents/cockatoos/data/accent_samples/recordings/recordings_standardized')
train_data = MFCCDataset(train_data_dir)

In [ ]:
for i in range(96):
    print(f"{i} {train_data[i].shape}")